## Overview
The whole processing has 2 steps:
1. Image classification: classifying images with or without ships. 
2. Image segmentation: segmenting ships from images.
We downsample images into 256 X 256. However, the downsampling caused ship size to be only 1 pixel, which leads to lower segmentation performance. So we select images with larger ship size for the segmentation.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai
from fastai.vision import *
from fastai.callbacks.hooks import *

import pandas as pd
import numpy as np
import os, glob

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

from azureml.core import Workspace, Datastore, Dataset, Experiment, Run, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

from azureml.train.dnn import PyTorch, Mpi
from azureml.train.hyperdrive import GridParameterSampling
from azureml.data.data_reference import DataReference
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice

from azureml.core.runconfig import MpiConfiguration

from azureml.widgets import RunDetails

### Prepare Azure Resource

In [ ]:
# Connect the workspace
interactive_auth = InteractiveLoginAuthentication()

subscription_id = '<Your Azure subscription id>'
resource_group = '<Your resource group in Azure'
workspace_name = '<Your workspace name>'

workspace = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name,
                      auth=interactive_auth)

In [ ]:
# Register storage container as datastore
storange_name = '<Your Azure storage name>'
ket_to_storage = '<Key to your storage>'
datastore_name = 'airbus'

datastore = Datastore.register_azure_blob_container(workspace=workspace, 
                                                    datastore_name=datastore_name, 
                                                    container_name=datastore_name,
                                                    account_name=storange_name, 
                                                    account_key=ket_to_storage,
                                                    create_if_not_exists=False)

In [ ]:
# Find datastore by name
datastore = Datastore.get(workspace, datastore_name)

In [ ]:
# Connect/create computer resource
cluster_name = 'gpu-nc24'

try:
    compute_target = ComputeTarget(workspace = workspace, name = cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_NC24', min_nodes = 0, max_nodes = 4)
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output = True, min_node_count = 4, timeout_in_minutes = 20)

In [ ]:
# Register dataset
dataset = Dataset.File.from_files(path=(datastore, 'airbus'))
dataset = dataset.register(workspace=workspace,
                           name='Airbus root',
                           description='Dataset for airbus images')

In [ ]:
# Define the script folder
script_folder = os.path.join(os.getcwd(), "training_scripts")

## Data Clean
1. downsize images to 256 X 256 
2. Put images to 2 folders: ship or no ship
3. Create the segmentation label images

In [ ]:
# Create experiment to clear data
exp_data = Experiment(workspace = workspace, name = 'urthecast_data_clean')

In [ ]:
# Register data reference
data_folder = DataReference(
    datastore=datastore,
    data_reference_name="airbus_root",
    path_on_datastore = 'airbus',
    mode = 'mount')

In [ ]:
# Create estimator for data clean
script_params = {
    '--data_folder': data_folder
}
est_data = PyTorch(source_directory = script_folder,
                    compute_target = compute_target,
                    entry_script = 'clean-data.py',  # python script for cleaning
                    script_params = script_params,
                    use_gpu = False,
                    node_count=1,
                    pip_packages = ['fastai'])

In [ ]:
# Submit for running
data_run = exp_data.submit(est_data)

In [ ]:
# Show run details
RunDetails(data_run).show()

## Ship/No ship classification

In [ ]:
# Create experiment to classification
exp_class = Experiment(workspace = workspace, name = 'classification')

In [ ]:
# Data reference for classification data
class_data_folder = DataReference(
    datastore=datastore,
    data_reference_name="airbus_class",
    path_on_datastore = 'airbus/class',
    mode = 'mount')

In [ ]:
# Estimator for classification
from azureml.train.dnn import PyTorch, Mpi

script_params = {
    '--data_folder': class_data_folder,
    '--num_epochs': 5
}

est_class = PyTorch(source_directory = script_folder,
                    compute_target = compute_target,
                    entry_script = 'classification.py', # Classification script
                    script_params = script_params,
                    use_gpu = True,
                    node_count=3,                       # 3 nodes are used
                    distributed_training=Mpi(process_count_per_node = 4), # 4 GPU's per node
                    pip_packages = ['fastai'])

In [ ]:
# Define the hyper drive for parameter tunning
param_sampling = GridParameterSampling({
    'start_learning_rate': choice(0.0001, 0.001),
    'end_learning_rate': choice(0.01, 0.1)})

hyperdrive_class = HyperDriveConfig(estimator = est_class,
                                         hyperparameter_sampling = param_sampling,
                                         policy = None,
                                         primary_metric_name = 'dice',
                                         primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs = 4,
                                         max_concurrent_runs = 4)

In [ ]:
# Kick off running
classification_run = exp_class.submit(hyperdrive_class)

In [ ]:
# Show running details
RunDetails(classification_run).show()

In [ ]:
# Get results for all running
classification_run.wait_for_completion(show_output = False)

children = list(classification_run.get_children())
metricslist = {}
i = 0

for single_run in children:
    results = {k: np.min(v) for k, v in single_run.get_metrics().items() if (k in ['dice', 'loss']) and isinstance(v, float)}
    parameters = single_run.get_details()['runDefinition']['arguments']
    try:
        results['start_learning_rate'] = parameters[5]
        results['end_learning_rate'] = parameters[7]
        metricslist[i] = results
        i += 1
    except:
        pass

rundata = pd.DataFrame(metricslist).sort_index(1).T.sort_values(by = ['loss'], ascending = True)
rundata

In [ ]:
# Show best running
best_run = classification_run.get_best_run_by_primary_metric()
best_run.get_file_names()

### Ship segmentation

In [ ]:
# Data reference for segmentation
sgmt_data_folder = DataReference(
    datastore=datastore,
    data_reference_name="airbus_segmentation",
    path_on_datastore = 'airbus/segmentation',
    mode = 'mount')

In [ ]:
# Experiment for segmentation
exp_sgmt = Experiment(workspace = workspace, name = 'segmentation')

In [ ]:
# Estimator for segmentation
segmt_script_params = {
    '--data_folder': sgmt_data_folder,
    '--img_folder': '256-filter99',
    '--num_epochs': 12
}

segmt_est = PyTorch(source_directory = script_folder,
                    compute_target = compute_target,
                    entry_script = 'segmentation.py', # Segmentation script
                    script_params = segmt_script_params,
                    use_gpu = True,
                    node_count=4,                     # 4 nodes
                    distributed_training=Mpi(process_count_per_node = 4), # 4 GPU's per node
                    pip_packages = ['fastai'])

In [ ]:
# Kick off running
segmentation_run = exp_sgmt.submit(config=segmt_est)

In [ ]:
# Running detail
RunDetails(segmentation_run).show()

In [ ]:
# Results
segmentation_run.wait_for_completion(show_output=False)  # specify True for a verbose log
print(segmentation_run.get_file_names())

In [ ]:
# Register model
model = larger_sgmt_run.register_model(model_name='segmentation-99',
                           tags={'ship': 'min99'},
                           model_path='outputs/segmentation.pkl')
print(model.name, model.id, model.version, sep='\t')

### Prediction
Sample code for prediction

In [ ]:
# Read image
size = 256
ifile = '<Test image>'
img = open_image(ifile)
img = img.resize(size)

In [ ]:
# Prediction
model_path = '<The model path>'
learn = load_learner(model_path)
pred = learn.predict(img)